In [5]:
# ✅ Step 1: Import Libraries
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend
output_notebook()


Loading BokehJS ...

In [7]:
# ✅ Load and preprocess with correct format
df = pd.read_csv("dataset.csv")

# Fix datetime format (dd-mm-yyyy HH:MM:SS)
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], dayfirst=True)

# Rename important columns
df = df.rename(columns={
    "SystemCodeNumber": "Lot_ID",
    "VehicleType": "Vehicle_Type",
    "TrafficConditionNearby": "Traffic_Level",
})

# Sort by Lot ID and Timestamp
df = df.sort_values(by=["Lot_ID", "Timestamp"])


In [10]:
# ✅ Step 3: Define Pricing Logic (Model 1 - Baseline Linear)

BASE_PRICE = 10.0
ALPHA = 5.0
MIN_PRICE = 5.0
MAX_PRICE = 20.0

# Store previous price per lot
prev_prices = {}

def pricing_logic(row):
    lot_id = row["Lot_ID"]
    occupancy = row["Occupancy"]
    capacity = row["Capacity"]

    prev_price = prev_prices.get(lot_id, BASE_PRICE)
    delta = ALPHA * (occupancy / capacity)
    new_price = prev_price + delta
    new_price = max(min(new_price, MAX_PRICE), MIN_PRICE)

    prev_prices[lot_id] = new_price
    return round(new_price, 2)


In [11]:
# ✅ Step 4: Apply Pricing Logic to Dataset
all_rows = []
for _, row in df.iterrows():
    price = pricing_logic(row)
    row["Price"] = price
    all_rows.append(row)

final_df = pd.DataFrame(all_rows)


In [12]:
# ✅ Step 5: Visualize Pricing Over Time with Bokeh
p = figure(x_axis_type="datetime", title="Model 1: Baseline Linear Pricing", width=950, height=450)
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'cyan', 'navy', 'olive', 'magenta', 'gray', 'teal', 'pink', 'black']
legend_items = []

lot_ids = final_df["Lot_ID"].unique()

for i, lot_id in enumerate(lot_ids):
    lot_df = final_df[final_df["Lot_ID"] == lot_id]
    line = p.line(x=lot_df["Timestamp"], y=lot_df["Price"], line_width=2, color=colors[i % len(colors)])
    legend_items.append((f"Lot {lot_id}", [line]))

legend = Legend(items=legend_items)
p.add_layout(legend, 'right')
p.xaxis.axis_label = "Timestamp"
p.yaxis.axis_label = "Price ($)"
show(p)


In [2]:
!pip install bokeh

  Using cached xyzservices-2025.4.0-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.0 MB 1.9 MB/s eta 0:00:04
   ---- ----------------------------------- 0.8/7.0 MB 1.5 MB/s eta 0:00:05
   ----- ---------------------------------- 1.0/7.0 MB 1.1 MB/s eta 0:00:06
   ------- -------------------------------- 1.3/7.0 MB 1.1 MB/s eta 0:00:06
   ------- -------------------------------- 1.3/7.0 MB 1.1 MB/s eta 0:00:06
   -------- ------------------------------- 1.6/7.0 MB 1.1 MB/s eta 0:00:05
   ---------- ----------------------------- 1.8/7.0 MB 1.1 MB/s eta 0:00:05
   ----------- ---------------------------- 2.1/7.0 MB 1.1 MB/s eta 0:00:05
   -------------- ------------------------- 2.6/7.0 MB 1.2 MB/s eta 0:00:04
   ----------------- -----------------